# Host-Pathogen Interaction Analysis: Klebsiella pneumoniae in Lung Macrophages

## Study Overview

This analysis examines the transcriptional response of lung macrophages to *Klebsiella pneumoniae* infection using single-cell RNA sequencing data from **GSE184290**. The study focuses on two distinct macrophage populations in the lung:

- **Alveolar Macrophages (AM)**: Resident macrophages in the alveolar space
- **Interstitial Macrophages (IM)**: Macrophages in the lung interstitium

## Experimental Design

**Conditions analyzed:**
- **KP+**: Cells infected with *Klebsiella pneumoniae*
- **KP-**: Uninfected cells from infected animals (bystander effect)
- **Control**: Cells from uninfected control animals

**Sample Groups:**
- `GSM5583415`: AM_KP- (Alveolar macrophages, uninfected from infected mice)
- `GSM5583416`: AM_KP+ (Alveolar macrophages, infected)
- `GSM5583417`: IM_KP- (Interstitial macrophages, uninfected from infected mice)
- `GSM5583418`: IM_KP+ (Interstitial macrophages, infected)
- `GSM5583419`: AM_Control (Alveolar macrophages, control)
- `GSM5583420`: IM_Control (Interstitial macrophages, control)

## Analysis Pipeline

This notebook performs the initial data construction and validation steps:

1. **Data Loading**: Import purified and downsampled count matrices
2. **Metadata Integration**: Fetch and merge sample metadata from GEO
3. **AnnData Construction**: Create standardized single-cell data object
4. **Data Validation**: Verify data integrity and structure

The resulting AnnData object will serve as input for downstream quality control, clustering, and differential expression analyses.

## Data Loading and Preprocessing

### Sample File Mapping

The analysis uses purified and downsampled count matrices for each sample. Each file contains gene expression counts for the respective macrophage population and treatment condition.

In [1]:
import pandas as pd
import anndata as ad
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import os
import GEOparse

sc.settings.verbosity = 1

In [2]:
sample_files = {
    'GSM5583415': 'data/purified/GSM5583415_PN0162_0001_counts_purified_downsampled.txt',  # AM_KP-
    'GSM5583416': 'data/purified/GSM5583416_PN0162_0002_counts_purified_downsampled.txt',  # AM_KP+
    'GSM5583417': 'data/purified/GSM5583417_PN0162_0003_counts_purified_downsampled.txt',  # IM_KP-
    'GSM5583418': 'data/purified/GSM5583418_PN0162_0004_counts_purified_downsampled.txt',  # IM_KP+
    'GSM5583419': 'data/purified/GSM5583419_PN0162_0005_counts_purified_downsampled.txt',  # AM_Control
    'GSM5583420': 'data/purified/GSM5583420_PN0162_0006_counts_purified_downsampled.txt',  # IM_Control
}

### Matrix Concatenation

Load and concatenate all sample count matrices into a single dataset. Each matrix is transposed from the original genes × cells format to the standard cells × genes format required by AnnData.

In [3]:
combined_data = []

for gsm_id, file_path in sample_files.items():
    # Load count matrix (genes × cells)
    counts_df = pd.read_csv(file_path, sep='\t', index_col=0)
    
    # Transpose to cells × genes format
    counts_transposed = counts_df.T
    
    # Add sample identifier for metadata tracking
    counts_transposed['sample_id'] = gsm_id
    
    combined_data.append(counts_transposed)
    print(f"Loaded {gsm_id}: {counts_transposed.shape[0]} cells × {counts_transposed.shape[1]-1} genes")

print(f"\nTotal samples processed: {len(combined_data)}")

Loaded GSM5583415: 422 cells × 15547 genes
Loaded GSM5583416: 422 cells × 15547 genes
Loaded GSM5583417: 422 cells × 15547 genes
Loaded GSM5583418: 422 cells × 15547 genes
Loaded GSM5583419: 422 cells × 15547 genes
Loaded GSM5583420: 422 cells × 15547 genes

Total samples processed: 6


In [4]:
# Concatenate all samples into single matrix
combined_counts = pd.concat(combined_data, axis=0, ignore_index=False)

# Separate sample metadata from count matrix
sample_metadata = combined_counts['sample_id'].copy()
count_matrix = combined_counts.drop(columns=['sample_id'])

print(f"Combined dataset shape: {count_matrix.shape[0]} cells × {count_matrix.shape[1]} genes")
print(f"Sample distribution:")
print(sample_metadata.value_counts().sort_index())

Combined dataset shape: 2532 cells × 15547 genes
Sample distribution:
sample_id
GSM5583415    422
GSM5583416    422
GSM5583417    422
GSM5583418    422
GSM5583419    422
GSM5583420    422
Name: count, dtype: int64


## Metadata Integration

### GEO Metadata Retrieval

Fetch comprehensive sample metadata from the Gene Expression Omnibus (GEO) database to enrich our dataset with experimental details.

In [5]:
# Download and parse GEO metadata
gse_id = "GSE184290"
download_dir = './metadata'
os.makedirs(download_dir, exist_ok=True)

gse = GEOparse.get_GEO(geo=gse_id, destdir=download_dir)

# Extract sample metadata
sample_metadata_list = []
for gsm_id, gsm_obj in gse.gsms.items():
    metadata = {key: value[0] if value else None for key, value in gsm_obj.metadata.items()}
    metadata['sample_id'] = gsm_id
    sample_metadata_list.append(metadata)

geo_metadata = pd.DataFrame(sample_metadata_list).set_index('sample_id')

print(f"Retrieved metadata for {len(geo_metadata)} samples")
print("Sample information:")
print(geo_metadata[['title', 'source_name_ch1', 'organism_ch1']].head())

06-Jul-2025 15:32:46 DEBUG utils - Directory ./metadata already exists. Skipping.
06-Jul-2025 15:32:46 INFO GEOparse - File already exist: using local version.
06-Jul-2025 15:32:46 INFO GEOparse - Parsing ./metadata/GSE184290_family.soft.gz: 
06-Jul-2025 15:32:46 DEBUG GEOparse - DATABASE: GeoMiame
06-Jul-2025 15:32:46 DEBUG GEOparse - SERIES: GSE184290
06-Jul-2025 15:32:46 DEBUG GEOparse - PLATFORM: GPL24247
06-Jul-2025 15:32:46 DEBUG GEOparse - SAMPLE: GSM5583415
06-Jul-2025 15:32:46 DEBUG GEOparse - SAMPLE: GSM5583416
06-Jul-2025 15:32:46 DEBUG GEOparse - SAMPLE: GSM5583417
06-Jul-2025 15:32:46 DEBUG GEOparse - SAMPLE: GSM5583418
06-Jul-2025 15:32:46 DEBUG GEOparse - SAMPLE: GSM5583419
06-Jul-2025 15:32:46 DEBUG GEOparse - SAMPLE: GSM5583420


Retrieved metadata for 6 samples
Sample information:
                 title source_name_ch1  organism_ch1
sample_id                                           
GSM5583415      AM_KP-            Lung  Mus musculus
GSM5583416      AM_KP+            Lung  Mus musculus
GSM5583417      IM_KP-            Lung  Mus musculus
GSM5583418      IM_KP+            Lung  Mus musculus
GSM5583419  AM_Control            Lung  Mus musculus


## AnnData Construction

### Create AnnData Object

Construct the standardized AnnData object with integrated count data and metadata.

In [6]:
# Create AnnData object
adata = ad.AnnData(count_matrix)

# Add sample metadata to observations
adata.obs['sample_id'] = sample_metadata.loc[adata.obs_names]

# Merge with GEO metadata
adata.obs = adata.obs.merge(
    geo_metadata,
    left_on='sample_id',
    right_index=True,
    how='left'
).set_index(adata.obs_names)

# Create cleaner column names for analysis
adata.obs['condition'] = adata.obs['title']

# Ensure unique gene names
adata.var_names_make_unique()

print(f"AnnData object created: {adata.shape[0]} cells × {adata.shape[1]} genes")
print(f"Metadata columns: {len(adata.obs.columns)}")
print(f"Experimental conditions: {sorted(adata.obs['condition'].unique())}")

AnnData object created: 2532 cells × 15547 genes
Metadata columns: 37
Experimental conditions: ['AM_Control', 'AM_KP+', 'AM_KP-', 'IM_Control', 'IM_KP+', 'IM_KP-']


In [7]:
# Save the constructed AnnData object
output_file = "GSE184290_data.h5ad"
adata.write(output_file)

print(f"AnnData object saved to: {output_file}")
print(f"Dataset summary:")
print(f"  - {adata.n_obs:,} cells")
print(f"  - {adata.n_vars:,} genes") 
print(f"  - {len(adata.obs['sample_id'].unique())} samples")
print(f"  - Conditions: {sorted(adata.obs['condition'].unique())}")

AnnData object saved to: GSE184290_data.h5ad
Dataset summary:
  - 2,532 cells
  - 15,547 genes
  - 6 samples
  - Conditions: ['AM_Control', 'AM_KP+', 'AM_KP-', 'IM_Control', 'IM_KP+', 'IM_KP-']


## Data Validation

### Load and Inspect AnnData Object

Verify the integrity and structure of the constructed dataset before proceeding with downstream analyses.

In [8]:
# Load the saved AnnData object
adata = sc.read_h5ad('GSE184290_data.h5ad')

print("=== Dataset Overview ===")
print(f"Data matrix type: {type(adata.X)}")
print(f"Shape: {adata.shape[0]:,} cells × {adata.shape[1]:,} genes")
print(f"Data type: {adata.X.dtype}")
print(f"Is sparse: {hasattr(adata.X, 'nnz')}")

print("\n=== Sample Distribution ===")
sample_counts = adata.obs['condition'].value_counts()
for condition, count in sample_counts.items():
    print(f"  {condition}: {count:,} cells")

print(f"\nTotal cells: {sample_counts.sum():,}")
print(f"Average cells per sample: {sample_counts.mean():.0f}")

=== Dataset Overview ===
Data matrix type: <class 'numpy.ndarray'>
Shape: 2,532 cells × 15,547 genes
Data type: int64
Is sparse: False

=== Sample Distribution ===
  AM_Control: 422 cells
  AM_KP+: 422 cells
  AM_KP-: 422 cells
  IM_Control: 422 cells
  IM_KP+: 422 cells
  IM_KP-: 422 cells

Total cells: 2,532
Average cells per sample: 422


### Data Quality Checks

Perform comprehensive quality checks on the count matrix to ensure data integrity.

In [9]:
print("=== Expression Matrix Statistics ===")
print(f"Minimum count: {adata.X.min()}")
print(f"Maximum count: {adata.X.max()}")
print(f"Mean count: {adata.X.mean():.3f}")
print(f"Standard deviation: {adata.X.std():.3f}")
print(f"Median count: {np.median(adata.X):.3f}")

print("\n=== Data Integrity Checks ===")
nan_check = np.isnan(adata.X).any()
inf_check = np.isinf(adata.X).any()
negative_check = (adata.X < 0).any()

print(f"Contains NaN values: {nan_check}")
print(f"Contains infinite values: {inf_check}")
print(f"Contains negative values: {negative_check}")

if nan_check or inf_check or negative_check:
    print("WARNING: Data integrity issues detected!")
else:
    print("Data integrity checks passed")

=== Expression Matrix Statistics ===
Minimum count: 0
Maximum count: 8956
Mean count: 1.046
Standard deviation: 12.608
Median count: 0.000

=== Data Integrity Checks ===
Contains NaN values: False
Contains infinite values: False
Contains negative values: False
Data integrity checks passed


### Metadata and Structure Validation

Verify that the AnnData object has the expected structure and metadata organization.

In [10]:
print("=== AnnData Structure ===")
print(f"Observations (cells): {adata.n_obs:,}")
print(f"Variables (genes): {adata.n_vars:,}")
print(f"Observation metadata columns: {adata.obs.shape[1]}")
print(f"Variable metadata columns: {adata.var.shape[1]}")

print(f"\nUnstructured annotations: {list(adata.uns.keys())}")
print(f"Observation matrices: {list(adata.obsm.keys())}")
print(f"Variable matrices: {list(adata.varm.keys())}")

print("\n=== Key Metadata Columns ===")
key_columns = ['sample_id', 'condition', 'organism_ch1', 'platform_id']
for col in key_columns:
    if col in adata.obs.columns:
        unique_vals = adata.obs[col].nunique()
        print(f"  {col}: {unique_vals} unique values")
    else:
        print(f"  {col}: not found")

print("\n=== Sample Metadata Preview ===")
display_cols = ['sample_id', 'condition', 'organism_ch1'] 
available_cols = [col for col in display_cols if col in adata.obs.columns]
if available_cols:
    print(adata.obs[available_cols].head())

=== AnnData Structure ===
Observations (cells): 2,532
Variables (genes): 15,547
Observation metadata columns: 37
Variable metadata columns: 0

Unstructured annotations: []
Observation matrices: []
Variable matrices: []

=== Key Metadata Columns ===
  sample_id: 6 unique values
  condition: 6 unique values
  organism_ch1: 1 unique values
  platform_id: 1 unique values

=== Sample Metadata Preview ===
                     sample_id condition  organism_ch1
TGCTTCGTCTGTACAG_1  GSM5583415    AM_KP-  Mus musculus
AGTAGCTAGTAACCTC_1  GSM5583415    AM_KP-  Mus musculus
GTGGAGAAGAGTACCG_1  GSM5583415    AM_KP-  Mus musculus
GTAATGCTCTCATTAC_1  GSM5583415    AM_KP-  Mus musculus
CGCATAAAGTCGAAAT_1  GSM5583415    AM_KP-  Mus musculus


In [11]:
print("=== Validation Summary ===")
print(f"Successfully constructed AnnData object")
print(f"{adata.n_obs:,} cells from {len(adata.obs['sample_id'].unique())} samples")
print(f"{adata.n_vars:,} genes with unique identifiers")
print(f"Comprehensive metadata integrated from GEO")
print(f"Data ready for quality control and downstream analysis")

print(f"\nOutput file: GSE184290_data.h5ad")
print(f"Next steps: Quality control, normalization, and clustering")

=== Validation Summary ===
Successfully constructed AnnData object
2,532 cells from 6 samples
15,547 genes with unique identifiers
Comprehensive metadata integrated from GEO
Data ready for quality control and downstream analysis

Output file: GSE184290_data.h5ad
Next steps: Quality control, normalization, and clustering
